In [123]:
from difflib import SequenceMatcher  
import numpy as np
import findspark
findspark.init()
import pyspark
from pyspark.sql import *
from pyspark import SparkContext, SparkConf

In [2]:
from platform import python_version

print(python_version())

3.5.2


In [3]:
# create the session
conf = SparkConf().setMaster("local").setAppName("My app")
# create the context
#sc = pyspark.SparkContext(conf=conf)
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

In [4]:
spark

# Function for comparing records with references

In [117]:
def func(values):

    return 1 if (SequenceMatcher(None, values[0].split(":")[1], values[1].split(":")[1]).ratio() > 0.8) else 0
     

# Contents of Input Files

In [5]:
inputfile = sc.textFile("./Input.txt")
referencefile = sc.textFile("./Reference.txt")

In [6]:
inputfile.collect()

['1:1 0 1 0 1 1 1 1 0 1 0',
 '2:1 0 1 0 1 1 1 1 0 0 0',
 '3:1 0 0 0 0 1 1 1 0 1 0',
 '4:1 1 1 1 1 1 1 1 0 1 1']

In [7]:
referencefile.collect()

['R1:1 0 0 0 1 1 1 1 0 0 0', 'R2:1 1 1 1 1 1 1 1 1 1 1']

# Cartesian Product

In [136]:
inputfile.cartesian(referencefile).collect()

[('1:1 0 1 0 1 1 1 1 0 1 0', 'R1:1 0 0 0 1 1 1 1 0 0 0'),
 ('1:1 0 1 0 1 1 1 1 0 1 0', 'R2:1 1 1 1 1 1 1 1 1 1 1'),
 ('2:1 0 1 0 1 1 1 1 0 0 0', 'R1:1 0 0 0 1 1 1 1 0 0 0'),
 ('2:1 0 1 0 1 1 1 1 0 0 0', 'R2:1 1 1 1 1 1 1 1 1 1 1'),
 ('3:1 0 0 0 0 1 1 1 0 1 0', 'R1:1 0 0 0 1 1 1 1 0 0 0'),
 ('3:1 0 0 0 0 1 1 1 0 1 0', 'R2:1 1 1 1 1 1 1 1 1 1 1'),
 ('4:1 1 1 1 1 1 1 1 0 1 1', 'R1:1 0 0 0 1 1 1 1 0 0 0'),
 ('4:1 1 1 1 1 1 1 1 0 1 1', 'R2:1 1 1 1 1 1 1 1 1 1 1')]

# Similarity Calculation using output of cartesian product

In [118]:
inputfile.cartesian(referencefile).map(func).collect()

[1, 0, 1, 0, 1, 0, 0, 1]

In [119]:
output = inputfile.cartesian(referencefile).map(func).collect()

# Display Output

In [135]:
j = 0

q = []

for i in range(len(output)):
    
    if j>0 and j%2==0:
        
        q.append(output[i])
        
        k = np.argmax(q)
        
        print (int(j/2)," => ","R"+str(k+1))
        
        q = []
        
        q.append(output[i])
        
    else:
        
        q.append(output[i])
        
    j+=1    

k = np.argmax(q)
        
print (int(j/2)," => ","R"+str(k+1))

1  =>  R1
2  =>  R1
3  =>  R1
4  =>  R2
